In [1]:
# !ls $FLINK_HOME/lib

In [2]:
from pyflink.table import DataTypes, TableEnvironment, EnvironmentSettings
from pyflink.table.expressions import lit

In [3]:
settings = EnvironmentSettings.new_instance().in_batch_mode().use_blink_planner().build()
t_env = TableEnvironment.create(settings)

In [4]:
content = "line Licensed to the Apache Software Foundation ASF under one " \
              "line or more contributor license agreements See the NOTICE file " \
              "line distributed with this work for additional information " \
              "line regarding copyright ownership The ASF licenses this file " \
              "to you under the Apache License Version the " \
              "License you may not use this file except in compliance " \
              "with the License"

with open('test.txt', 'w') as fopen:
    fopen.write('\n'.join(content.split()))

In [5]:
my_source_ddl = """
    create table mySource (
        word VARCHAR
    ) with (
        'connector' = 'filesystem',
        'format' = 'csv',
        'path' = 'test.txt'
    )
"""

"""
create table mysink (word varchar, count bigint);
"""

my_sink_ddl = f"""
    create table mySink (
        word VARCHAR,
        `count` BIGINT
    ) with (
        'connector' = 'jdbc',
        'url' = 'jdbc:postgresql://postgres:5432/postgres',
        'table-name' = 'mysink',
        'username' = 'postgres',
        'password' = 'postgres'
    )
"""

t_env.execute_sql(my_source_ddl)
t_env.execute_sql(my_sink_ddl)

In [6]:
tab = t_env.from_path('mySource')
tab

In [7]:
tab.to_pandas().head()

,word
0,line
1,regarding
2,copyright
3,ownership
4,The


In [8]:
tab.group_by(tab.word) \
   .select(tab.word, lit(1).count) \
   .execute_insert('mySink').wait()

In [9]:
tab = t_env.from_path('mySink')
tab.to_pandas()

,word,count
0,regarding,1
1,copyright,1
2,ownership,1
3,ASF,2
4,this,3
5,to,2
6,under,2
7,License,3
8,Licensed,1
9,Foundation,1
